# Analisis de acciones Bancolombia

# 2020-01-01 - 2020-01-01

In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import yfinance as yf
import numpy as np
import datetime as dt
import warnings
import matplotlib.pyplot as plt


warnings.filterwarnings("ignore")

## 0. Introducción e importación de datos

La elección de la base de datos que abarca el periodo 2000-01-01 a 2020-01-01 para analizar las acciones de Bancolombia se justifica por la exclusión del impacto de la pandemia mundial, permitiendo un enfoque claro en factores y tendencias previas a este suceso. La extensión temporal facilita la evaluación del comportamiento de las acciones, buscando determinar si siguen una caminata aleatoria, lo que es esencial para evaluar la eficacia de estrategias de inversión basadas en análisis histórico. Además, la serie temporal presenta irregularidades en la espaciación de los datos, lo que agrega complejidad al análisis y requiere adaptación en las técnicas utilizadas para tener en cuenta la variabilidad en la frecuencia de observaciones.

In [2]:
ticker_name = 'BC'

data = yf.download(ticker_name, start='2000-01-01', end='2020-01-01')
print(data.shape)
data.head()

[*********************100%***********************]  1 of 1 completed
(5031, 6)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,22.0000,22.09375,21.6875,21.9375,15.422204,286700
2000-01-04,21.9375,22.12500,20.8750,20.9375,14.719192,338700
2000-01-05,20.6875,21.62500,20.6875,21.0625,14.807070,214000
2000-01-06,21.1250,21.31250,20.8750,21.2500,14.938889,232700
2000-01-07,21.2500,21.62500,20.9375,21.3750,15.026759,207800


In [3]:
data.shape

(5031, 6)

In [4]:
df = pd.DataFrame()
df["Date"] = pd.to_datetime(data.index).date
df["Close"] = data["Close"].values
df.head()

,Date,Close
0,2000-01-03,21.9375
1,2000-01-04,20.9375
2,2000-01-05,21.0625
3,2000-01-06,21.2500
4,2000-01-07,21.3750


In [5]:

fig = go.Figure()
# Suponiendo que ya tienes el DataFrame df

fig.add_trace(go.Scatter(
    x=df["Date"],
    y=df["Close"],
    name="NeuralProphet Forescasting",
    line=dict(color='blue', width=2)
))
fig.update_xaxes(type='category')  # Para que las fechas se muestren correctamente

# Centrar el título
fig.update_layout(title_text='Precio de acciones Bancolombia', title_x=0.5)
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1d",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="1w",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=3,
                     label="3m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    ),
    yaxis=dict(
        autorange=True,
        type="linear"
    )
)

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Price",
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=-.35
    )
)

# Análisis descriptivo de una serie de tiempo

## 1. Estabilizar varianza.
En esta sección se buscará una herramienta para estabilizar la varianza, siempre y cuando pareciera que no se encuentre estabilizada.

#### Familia de transformaciones Box-Cox:

En ocasiones la serie presenta varianza marginal no constante a lo largo del tiempo, lo cual hace necesario tener en cuenta tal característica. En este caso, se siguiere hacer una transformación de potencia para estabilizar la varianza. Esta familia de transformaciones se llaman transformaciones Box-Cox.

$$
    f_{\lambda}(u_{t})= \begin{cases}
        \lambda^{-1}(u^{\lambda}_{t}-1), &  \text{si  $u_{t} \geq 0$, para $\lambda>0$,}\\
        \ln(u_{t}), &\text{ si $u_{t}>0$, para $\lambda=0$}.
    \end{cases}
$$

note que la familia de funciones dependen del $\lambda$ escogido, pero como se escoge $\lambda$?. La respuesta es usando librerías de $R$ o Python.

**Nota**: Si el lambda calculado es muy cercano a 1, se recomienda no hacer Box-cox

In [10]:
from scipy.stats import boxcox

In [54]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objects as go
import yfinance as yf
import numpy as np
from scipy.stats import boxcox
import warnings

warnings.filterwarnings("ignore")

# Importar datos
ticker_name = 'BC'
data = yf.download(ticker_name, start='2000-01-01', end='2020-01-01')
df = pd.DataFrame()
df["Date"] = pd.to_datetime(data.index).date
df["Close"] = data["Close"].values

# Análisis Box-Cox
df['BoxCox_Close'], lambda_value = boxcox(df['Close'])  # Agregar 1 para evitar problemas con valores no positivos

fig = go.Figure()
# Suponiendo que ya tienes el DataFrame df

fig.add_trace(go.Scatter(
    x=df["Date"],
    y=df["Close"],
    name="Original",
    line=dict(color='blue', width=2)
))
fig.add_trace(go.Scatter(
    x=df["Date"],
    y=df["BoxCox_Close"],
    name="Box-Cox",
    line=dict(color='green', width=2)
))
fig.update_xaxes(type='category')  # Para que las fechas se muestren correctamente

# Centrar el título
fig.update_layout(title=dict(
        text=f'Precio de acciones Bancolombia \n Box-Cox (λ={round(lambda_value, 3)})',
        font=dict(size=20)  # Ajustar el tamaño del título
    ) , title_x=0.75)
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1d",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="1w",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=3,
                     label="3m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    ),
    yaxis=dict(
        autorange=True,
        type="linear"
    )
)

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Price",
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=-.35
    ),
    margin=dict(l=0, r=20, t=100, b=50)
)

fig.update_layout(
    xaxis=dict(title=dict(font=dict(size=20))),
    yaxis=dict(title=dict(font=dict(size=20))),
    legend=dict(font=dict(size=17))
)



[*********************100%***********************]  1 of 1 completed


In [12]:
df

,Date,Close,BoxCox_Close
0,2000-01-03,21.937500,10.947292
1,2000-01-04,20.937500,10.550359
2,2000-01-05,21.062500,10.600283
3,2000-01-06,21.250000,10.675003
4,2000-01-07,21.375000,10.724705
...,...,...,...
5026,2019-12-24,61.270000,23.935547
5027,2019-12-26,60.700001,23.770466
5028,2019-12-27,60.779999,23.793662
5029,2019-12-30,60.110001,23.599098
